<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python**

In [14]:
%%writefile tp2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validate_args(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(
          min=MIN_MATRIX_SIZE,
          max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrix_generator(size):
    matrix_result=[]
    for i in range(0, size):
        matrix_row=[]
        for j in range(0, size):
            matrix_row.append(random.randint(
              MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrix_result.append(matrix_row)
    return matrix_result

def print_matrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuential_mult(size, matrix_a, matrix_b):
    matrix_result = []
    for i in range(0, size):
        row_result = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                row_result[j] = matrix_a[i][k] * matrix_b[k][j]
        matrix_result.append(row_result)
    return matrix_result

## Funcion que ejecuta el thread
def multiply_row(size, matrix_a, matrix_b, row_result, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            row_result[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

## Genera un thread por cada fila encargado de ejecutar la multiplicacion y generar la fila resultante
def concurrent_mult(size, matrix_a, matrix_b):
    matrix_result = []
    threads = []

    for i in range(0, size):
        row_result = [None] * size
        thread = Thread(
          target=multiply_row,
          args=(size, matrix_a, matrix_b, row_result, i))
        thread.start()
        threads.append(thread)
        matrix_result.append(row_result)

    for thread in threads:
        thread.join()

    return matrix_result

def compare_matrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validate_args(sys.argv)
    matrix_a = matrix_generator(size)
    matrix_b = matrix_generator(size)
    matrix_secuential = secuential_mult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrent_mult(size,matrix_a,matrix_b)

    print_matrix("A", matrix_a)
    print_matrix("B", matrix_b)
    print_matrix("Secuencial", matrix_secuential)
    print_matrix("Concurrente", matrix_concurrent)

    ## Determinamos si las matrices son iguales
    result_compare = (
      "iguales" 
      if compare_matrix(size, matrix_secuential, matrix_concurrent)
      else "distintas")
    print("Las matrices concurrente y secuencial son {compare}".format(
      compare=result_compare))

main()

Overwriting tp2.py


In [15]:
!python3 tp2.py 1

El tamaño debe estar entre 5 y 20


In [16]:
!python3 tp2.py 7


Matrix: A
	16		-5		-1		3		-22		-5		10	
	-12		-25		13		26		-5		-15		-24	
	-25		-12		-16		11		-14		4		-32	
	22		29		-20		29		-15		11		13	
	-5		-28		15		20		4		-6		-26	
	-4		15		-29		29		25		-20		-27	
	-3		28		1		25		12		-13		22	

Matrix: B
	-17		18		-9		-2		9		-31		23	
	-18		-32		6		15		15		-5		9	
	-26		-11		-8		15		13		-25		23	
	27		-14		-9		18		9		-13		14	
	5		-30		9		13		-30		21		6	
	-22		28		24		-18		27		3		12	
	3		0		6		-24		-31		-4		30	

Matrix: Secuencial
	30		0		60		-240		-310		-40		300	
	-72		0		-144		576		744		96		-720	
	-96		0		-192		768		992		128		-960	
	39		0		78		-312		-403		-52		390	
	-78		0		-156		624		806		104		-780	
	-81		0		-162		648		837		108		-810	
	66		0		132		-528		-682		-88		660	

Matrix: Concurrente
	30		0		60		-240		-310		-40		300	
	-72		0		-144		576		744		96		-720	
	-96		0		-192		768		992		128		-960	
	39		0		78		-312		-403		-52		390	
	-78		0		-156		624		806		104		-780	
	-81		0		-162		648		837		108		-810	
	66		0		132		-528		-682		-88		660	
Las matrices concurre